In [1]:
!python -V

Python 3.12.3


In [12]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [8]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment-mlops")

2026/01/17 10:18:43 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment-mlops' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/MLops/experiment_tracking/mlruns/2', creation_time=1768645123600, experiment_id='2', last_update_time=1768645123600, lifecycle_stage='active', name='nyc-taxi-experiment-mlops', tags={}>

In [21]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df
    

In [22]:
df_train = read_dataframe('/workspaces/MLops/experiment_tracking/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/workspaces/MLops/experiment_tracking/green_tripdata_2021-02.parquet')

In [24]:
categorical = ['duration'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [31]:
from sklearn.metrics import mean_squared_error
y_train = df_train['duration'].values
y_val = df_val['duration'].values
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE: {rmse:.2f} minutes")

RMSE: 0.00 minutes


In [32]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [34]:
import numpy as np
import mlflow

with mlflow.start_run():
    mlflow.set_tag("developer", "Tom")
    mlflow.log_param("train_data_path", "green_tripdata_2021-01.parquet")
    mlflow.log_param("valid_data_path", "green_tripdata_2021-02.parquet")
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    
    # Calculate RMSE without squared parameter
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)
    
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")
    
    print(f"RMSE: {rmse:.2f} minutes")

RMSE: 0.00 minutes


In [35]:
import xgboost as xgb

In [36]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/workspaces/MLops/.venv/lib/python3.12/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [37]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [43]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
import numpy as np
import mlflow
import xgboost as xgb
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
from sklearn.metrics import mean_squared_error


train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)


def objective(params):
    with mlflow.start_run(nested=True):
        mlflow.log_params(params)
        
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10,
            verbose_eval=False
        )
        
        y_pred = booster.predict(valid)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        
        mlflow.log_metric("rmse", rmse)
    
    return {'loss': rmse, 'status': STATUS_OK}


search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}


with mlflow.start_run():
    best_result = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=50,
        trials=Trials()
    )
    
    print(f"Best parameters: {best_result}")

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [00:41<00:00,  1.21trial/s, best loss: 0.16381026102297552]
Best parameters: {'learning_rate': np.float64(0.19439163286173894), 'max_depth': np.float64(5.0), 'min_child_weight': np.float64(1.9233745276024308), 'reg_alpha': np.float64(0.3448767253571542), 'reg_lambda': np.float64(0.05266998547643627)}


In [50]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("/workspaces/MLops/experiment_tracking/models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=np.sqrt())
        mlflow.log_metric("rmse", rmse)
        

TypeError: sqrt() takes from 1 to 2 positional arguments but 0 were given

In [ ]:
import numpy as np
import mlflow
import pickle
import os

os.makedirs("models", exist_ok=True)


with open("models/preprocessor.b", "wb") as f_out:
    pickle.dump(dv, f_out)

with mlflow.start_run():
    mlflow.set_tag("developer", "Tom")
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
    
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    
    mlmodel = model_class()
    mlmodel.fit(X_train, y_train)
    
    y_pred = mlmodel.predict(X_val)
    
    # CORRECT way:
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    
    mlflow.log_metric("rmse", rmse)
    
    print(f"RMSE: {rmse:.4f}")

RMSE: 0.0020


In [52]:
params={
    'bootstrap': True,
                 
'ccp_alpha':0.0,

'criterion':'squared_error',

'max_depth':None,

'max_features':1.0,

'max_leaf_nodes':None,

'max_samples':None,

'min_impurity_decrease':0.0,

'min_samples_leaf':1,

'min_samples_split':2,

'min_weight_fraction_leaf':0.0,

'monotonic_cst':None,
'n_estimators':100,
'n_jobs':None,
'oob_score':False,
'random_state':None,
'train-data-path':'./data/green_tripdata_2021-01.csv',
'valid-data-path':'./data/green_tripdata_2021-02.csv',
'verbose':0,
'warm_start':False
}

mlflow.xgboost.autolog()
        
        
booster = xgb.train(
params=params,
dtrain=train,
num_boost_round=100,
evals=[(valid, 'validation')],
early_stopping_rounds=10,
verbose_eval=False
        )
        


2026/01/17 11:25:00 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '3c49e4fd84a54988a9530d60ac99a1a1', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/workspaces/MLops/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [11:25:00] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "bootstrap", "ccp_alpha", "criterion", "max_features", "min_impurity_decrease", "min_samples_leaf", "min_samples_split", "min_weight_fraction_leaf", "n_estimators", "oob_score", "train-data-path", "valid-data-path", "verbose", "warm_start" } are not used.

  self.starting_round = model.num_boosted_rounds()
2026/01/17 11:25:00 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2026/01/17 11:25:00 WARNING mlflow.models.model: `artifact_path` is deprec